In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt  

random_seed = 0

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor, StackingRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import BayesianRidge, SGDRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, StackingRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=random_seed)

models = {
    'Linear': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'RF': RandomForestRegressor(n_estimators=100, random_state=random_seed),
    'SVR': SVR(),
    'DT': DecisionTreeRegressor(random_state=random_seed),
    'GraBoost': GradientBoostingRegressor(random_state=random_seed),
    'AdaBoost': AdaBoostRegressor(random_state=random_seed),
    'XGBoost': XGBRegressor(random_state=random_seed),
    'Extra_Trees': ExtraTreesRegressor(n_estimators=100, random_state=random_seed),
    'KNN': KNeighborsRegressor(),
    'ElasticNet': ElasticNet(),
    'Stacking': StackingRegressor(
        estimators=[
            ('lr', Ridge()),
            ('dt', DecisionTreeRegressor(random_state=random_seed)),
            ('svr', make_pipeline(StandardScaler(), SVR()))
        ],
        final_estimator=LinearRegression(),
       # random_state=random_seed
    ),
    'Bagging': BaggingRegressor(estimator=DecisionTreeRegressor(random_state=random_seed), n_estimators=10, random_state=random_seed),
    'Bayesian Ridge': BayesianRidge(),
    'SGD': SGDRegressor(random_state=random_seed)
}

In [3]:
combined_df = pd.read_excel('/home/lic154/data/AttentiveFP-master/data_my/embs_combined_num_30+admet.xlsx', sheet_name='all')
combined_df.info()

# col 38-52:  13 additional features generated from ADMET predictor

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 53 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   HLM            301 non-null    float64
 1   KSOL           365 non-null    float64
 2   LogD           352 non-null    float64
 3   MDR1-MDCKII    425 non-null    float64
 4   MLM            303 non-null    float64
 5   Molecule Name  560 non-null    object 
 6   Dataset        560 non-null    object 
 7   cano_smiles    560 non-null    object 
 8   lipop_1        560 non-null    float64
 9   lipop_2        560 non-null    float64
 10  lipop_3        560 non-null    float64
 11  lipop_4        560 non-null    float64
 12  lipop_5        560 non-null    float64
 13  lipop_6        560 non-null    float64
 14  lipop_7        560 non-null    float64
 15  lipop_8        560 non-null    float64
 16  lipop_9        560 non-null    float64
 17  lipop_10       560 non-null    float64
 18  free_solv_

# ML training for Solubility

In [4]:
task = 'KSOL'

fp_KSOL_test = pd.read_csv('../data_my/fp_admet/dissect_KSOL_test.tsv', sep=r'\t|\s+', engine='python', header=None )
fp_KSOL_train = pd.read_csv('../data_my/fp_admet/dissect_KSOL_train.tsv', sep =r'\s+', engine='python', header=None)

fp_column_name =  ['fp_' + str(i+1) for i in range(len(fp_KSOL_test.columns)-1 )]

fp_KSOL_test.columns = ['Molecule Name'] + fp_column_name
fp_KSOL_train.columns =['Molecule Name'] + ['outputs'] + fp_column_name


tmp_KSOL_train = pd.merge( fp_KSOL_train, combined_df, on='Molecule Name', how='left')
train_KSOL = tmp_KSOL_train.drop(combined_df.columns[[0, 1, 2, 3, 4, 6, 7]], axis=1) 

tmp_KSOL_test = pd.merge(fp_KSOL_test, combined_df,  on='Molecule Name', how='left')
test_KSOL = tmp_KSOL_test.drop(combined_df.columns[[0, 1, 2, 3, 4, 6, 7]], axis=1) 

train_KSOL.dropna(inplace=True)
test_KSOL.dropna(inplace=True)

X_train = train_KSOL.iloc[:,2:]
y_train = train_KSOL['outputs']

X_test = test_KSOL.iloc[:,1:]


X_train.shape, X_test.shape

((365, 123), (126, 123))

## remove the FP desc

In [5]:
fp_KSOL_train.columns

Index(['Molecule Name', 'outputs', 'fp_1', 'fp_2', 'fp_3', 'fp_4', 'fp_5',
       'fp_6', 'fp_7', 'fp_8', 'fp_9', 'fp_10', 'fp_11', 'fp_12', 'fp_13',
       'fp_14', 'fp_15', 'fp_16', 'fp_17', 'fp_18', 'fp_19', 'fp_20', 'fp_21',
       'fp_22', 'fp_23', 'fp_24', 'fp_25', 'fp_26', 'fp_27', 'fp_28', 'fp_29',
       'fp_30', 'fp_31', 'fp_32', 'fp_33', 'fp_34', 'fp_35', 'fp_36', 'fp_37',
       'fp_38', 'fp_39', 'fp_40', 'fp_41', 'fp_42', 'fp_43', 'fp_44', 'fp_45',
       'fp_46', 'fp_47', 'fp_48', 'fp_49', 'fp_50', 'fp_51', 'fp_52', 'fp_53',
       'fp_54', 'fp_55', 'fp_56', 'fp_57', 'fp_58', 'fp_59', 'fp_60', 'fp_61',
       'fp_62', 'fp_63', 'fp_64', 'fp_65', 'fp_66', 'fp_67', 'fp_68', 'fp_69',
       'fp_70', 'fp_71', 'fp_72', 'fp_73', 'fp_74', 'fp_75', 'fp_76', 'fp_77',
       'fp_78'],
      dtype='object')

In [6]:
X_train = train_KSOL.drop(fp_KSOL_train.columns, axis = 1)
y_train = train_KSOL['outputs']

X_test = test_KSOL.drop(fp_KSOL_test.columns, axis=1)

X_train.shape, X_test.shape

((365, 45), (126, 45))

In [7]:
X_train.columns

Index(['lipop_1', 'lipop_2', 'lipop_3', 'lipop_4', 'lipop_5', 'lipop_6',
       'lipop_7', 'lipop_8', 'lipop_9', 'lipop_10', 'free_solv_1',
       'free_solv_2', 'free_solv_3', 'free_solv_4', 'free_solv_5',
       'free_solv_6', 'free_solv_7', 'free_solv_8', 'free_solv_9',
       'free_solv_10', 'esol_1', 'esol_2', 'esol_3', 'esol_4', 'esol_5',
       'esol_6', 'esol_7', 'esol_8', 'esol_9', 'esol_10', 'AP_FWeight',
       'DiffCoef', 'MlogP', 'S+logP', 'S+logD', 'S+MDCK', 'logHLC', 'S+Peff',
       'S+Sw', 'S+S_Intrins', 'S+MDCK-LE', 'S+S_pH', 'S+FaSSGF', 'S+FaSSIF',
       'S+FeSSIF'],
      dtype='object')

In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
# 归一化处理
scaler = MinMaxScaler()
#scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train)  # 用训练数据拟合并转换
X_test = scaler.transform(X_test)  # 用相同的缩放器转换测试数据

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score

metric_dict = {}

for name, model in models.items():
    print(f"Processing model: {name}")
    
    # 基于交叉验证计算各项指标（注意：sklearn的负值评分，所以需要取负）
    cv_mse = cross_val_score(model, X_train, y_train, cv=cv_strategy, scoring='neg_mean_squared_error')
    cv_mae = cross_val_score(model, X_train, y_train, cv=cv_strategy, scoring='neg_mean_absolute_error')
    cv_r2  = cross_val_score(model, X_train, y_train, cv=cv_strategy, scoring='r2')
    
    # 计算每个CV折中的RMSE（注意：cv_mse为负值）
    cv_rmse = np.sqrt(-cv_mse)
    
    # 如果需要，也可以在这里对整个训练集进行模型拟合（但不参与指标计算）
    model.fit(X_train, y_train)
    
    # 保存交叉验证指标的均值和标准差（作为变异性的度量）
    metric_dict[name] = {
        "CV MAE Mean": -np.mean(cv_mae),
        "CV MAE Std": np.std(cv_mae),
        "CV MSE Mean": -np.mean(cv_mse),
        "CV MSE Std": np.std(cv_mse),
        "CV RMSE Mean": np.mean(cv_rmse),
        "CV RMSE Std": np.std(cv_rmse),
        "CV R2 Mean": np.mean(cv_r2),
        "CV R2 Std": np.std(cv_r2)
    }
    
    print(f"Completed processing model: {name}\n")

# 将结果字典转换为DataFrame以便更好地查看
results_df = pd.DataFrame(metric_dict).T  # 转置，使得每一行对应一个模型
print(results_df)

Processing model: Linear
Completed processing model: Linear

Processing model: Ridge
Completed processing model: Ridge

Processing model: Lasso
Completed processing model: Lasso

Processing model: RF
Completed processing model: RF

Processing model: SVR
Completed processing model: SVR

Processing model: DT
Completed processing model: DT

Processing model: GraBoost
Completed processing model: GraBoost

Processing model: AdaBoost
Completed processing model: AdaBoost

Processing model: XGBoost
Completed processing model: XGBoost

Processing model: Extra_Trees
Completed processing model: Extra_Trees

Processing model: KNN
Completed processing model: KNN

Processing model: ElasticNet
Completed processing model: ElasticNet

Processing model: Stacking
Completed processing model: Stacking

Processing model: Bagging
Completed processing model: Bagging

Processing model: Bayesian Ridge
Completed processing model: Bayesian Ridge

Processing model: SGD
Completed processing model: SGD

            

In [10]:
results_df.sort_values('CV RMSE Mean', ascending=True).head()

,CV MAE Mean,CV MAE Std,CV MSE Mean,CV MSE Std,CV RMSE Mean,CV RMSE Std,CV R2 Mean,CV R2 Std
Extra_Trees,68.115376,8.801647,8530.263284,1941.973877,91.675343,11.220284,0.647922,0.092374
GraBoost,70.841131,12.393107,9392.089113,2379.116103,96.025979,13.080537,0.610848,0.115078
RF,75.139199,7.467877,9674.257607,1695.310379,97.934983,9.110252,0.601067,0.086105
XGBoost,69.496570,13.473667,10202.433112,2858.220296,99.922650,14.761337,0.577681,0.132826
KNN,72.864767,10.042044,10545.803215,2706.630027,101.850154,13.128192,0.565271,0.124556


In [11]:
results_df.sort_values('CV R2 Mean', ascending=False).head()

,CV MAE Mean,CV MAE Std,CV MSE Mean,CV MSE Std,CV RMSE Mean,CV RMSE Std,CV R2 Mean,CV R2 Std
Extra_Trees,68.115376,8.801647,8530.263284,1941.973877,91.675343,11.220284,0.647922,0.092374
GraBoost,70.841131,12.393107,9392.089113,2379.116103,96.025979,13.080537,0.610848,0.115078
RF,75.139199,7.467877,9674.257607,1695.310379,97.934983,9.110252,0.601067,0.086105
XGBoost,69.496570,13.473667,10202.433112,2858.220296,99.922650,14.761337,0.577681,0.132826
KNN,72.864767,10.042044,10545.803215,2706.630027,101.850154,13.128192,0.565271,0.124556


In [12]:
df_predictions_KSOL = pd.DataFrame( models['GraBoost'].predict(X_test), 
                              index=test_KSOL['Molecule Name'], 
                              columns=['KSOL']).reset_index()
print(df_predictions_KSOL.head())

  Molecule Name        KSOL
0  ASAP-0014619  375.441614
1  ASAP-0014632  390.752946
2  ASAP-0014717  349.858636
3  ASAP-0014745  352.542892
4  ASAP-0014750  232.484676


# ML training for MDCK-permeability

In [13]:
task = 'MDCK'

fp_MDCK_test = pd.read_csv('../data_my/fp_admet/dissect_MDCK_test.tsv', sep=r'\t|\s+', engine='python', header=None )
fp_MDCK_train = pd.read_csv('../data_my/fp_admet/dissect_MDCK_train.tsv', sep =r'\s+', engine='python', header=None)

fp_column_name =  ['fp_' + str(i+1) for i in range(len(fp_MDCK_test.columns)-1 )]

fp_MDCK_test.columns = ['Molecule Name'] + fp_column_name
fp_MDCK_train.columns =['Molecule Name'] + ['outputs'] + fp_column_name


tmp_MDCK_train = pd.merge( fp_MDCK_train, combined_df, on='Molecule Name', how='left')
train_MDCK = tmp_MDCK_train.drop(combined_df.columns[[0, 1, 2, 3, 4, 6, 7]], axis=1) 

tmp_MDCK_test = pd.merge(fp_MDCK_test, combined_df,  on='Molecule Name', how='left')
test_MDCK = tmp_MDCK_test.drop(combined_df.columns[[0, 1, 2, 3, 4, 6, 7]], axis=1) 


train_MDCK.dropna(inplace=True)
test_MDCK.dropna(inplace=True)

X_train = train_MDCK.iloc[:,2:]
y_train = train_MDCK['outputs']

X_test = test_MDCK.iloc[:,1:]

X_train.shape, X_test.shape

((424, 98), (126, 98))

In [14]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
# 归一化处理
scaler = MinMaxScaler()
#scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train)  # 用训练数据拟合并转换
# X_test = scaler.transform(X_test)  # 用相同的缩放器转换测试数据

In [15]:
metric_dict = {}

for name, model in models.items():
    print(f"Processing model: {name}")
    
    # 基于交叉验证计算各项指标（注意：sklearn的负值评分，所以需要取负）
    cv_mse = cross_val_score(model, X_train, y_train, cv=cv_strategy, scoring='neg_mean_squared_error')
    cv_mae = cross_val_score(model, X_train, y_train, cv=cv_strategy, scoring='neg_mean_absolute_error')
    cv_r2  = cross_val_score(model, X_train, y_train, cv=cv_strategy, scoring='r2')
    
    # 计算每个CV折中的RMSE（注意：cv_mse为负值）
    cv_rmse = np.sqrt(-cv_mse)
    
    # 如果需要，也可以在这里对整个训练集进行模型拟合（但不参与指标计算）
    model.fit(X_train, y_train)
    
    # 保存交叉验证指标的均值和标准差（作为变异性的度量）
    metric_dict[name] = {
        "CV MAE Mean": -np.mean(cv_mae),
        "CV MAE Std": np.std(cv_mae),
        "CV MSE Mean": -np.mean(cv_mse),
        "CV MSE Std": np.std(cv_mse),
        "CV RMSE Mean": np.mean(cv_rmse),
        "CV RMSE Std": np.std(cv_rmse),
        "CV R2 Mean": np.mean(cv_r2),
        "CV R2 Std": np.std(cv_r2)
    }
    
    print(f"Completed processing model: {name}\n")

# 将结果字典转换为DataFrame以便更好地查看
results_df = pd.DataFrame(metric_dict).T  # 转置，使得每一行对应一个模型
print(results_df)

Processing model: Linear
Completed processing model: Linear

Processing model: Ridge
Completed processing model: Ridge

Processing model: Lasso
Completed processing model: Lasso

Processing model: RF


Completed processing model: RF

Processing model: SVR
Completed processing model: SVR

Processing model: DT
Completed processing model: DT

Processing model: GraBoost
Completed processing model: GraBoost

Processing model: AdaBoost
Completed processing model: AdaBoost

Processing model: XGBoost
Completed processing model: XGBoost

Processing model: Extra_Trees
Completed processing model: Extra_Trees

Processing model: KNN
Completed processing model: KNN

Processing model: ElasticNet
Completed processing model: ElasticNet

Processing model: Stacking
Completed processing model: Stacking

Processing model: Bagging
Completed processing model: Bagging

Processing model: Bayesian Ridge
Completed processing model: Bayesian Ridge

Processing model: SGD
Completed processing model: SGD

                 CV MAE Mean    CV MAE Std   CV MSE Mean    CV MSE Std  \
Linear          5.001798e+11  4.792471e+11  3.809245e+25  5.832093e+25   
Ridge           3.169852e+00  3.120527e-01  2.242092e+01  4.3985

In [16]:
results_df.sort_values('CV RMSE Mean', ascending=True).head()

,CV MAE Mean,CV MAE Std,CV MSE Mean,CV MSE Std,CV RMSE Mean,CV RMSE Std,CV R2 Mean,CV R2 Std
Extra_Trees,2.191062,0.127700,11.866092,2.125009,3.431143,0.305534,0.704501,0.068119
GraBoost,2.334461,0.056514,14.165770,2.792817,3.746804,0.356690,0.646500,0.089112
XGBoost,2.424997,0.161560,15.365432,1.552770,3.914827,0.198904,0.618655,0.064638
RF,2.549033,0.106106,16.188269,1.507316,4.018881,0.191991,0.591793,0.103506
Bagging,2.604050,0.181685,17.482760,2.177780,4.172762,0.266121,0.556698,0.128074


In [17]:
results_df.sort_values('CV R2 Mean', ascending=False).head()

,CV MAE Mean,CV MAE Std,CV MSE Mean,CV MSE Std,CV RMSE Mean,CV RMSE Std,CV R2 Mean,CV R2 Std
Extra_Trees,2.191062,0.127700,11.866092,2.125009,3.431143,0.305534,0.704501,0.068119
GraBoost,2.334461,0.056514,14.165770,2.792817,3.746804,0.356690,0.646500,0.089112
XGBoost,2.424997,0.161560,15.365432,1.552770,3.914827,0.198904,0.618655,0.064638
RF,2.549033,0.106106,16.188269,1.507316,4.018881,0.191991,0.591793,0.103506
Bagging,2.604050,0.181685,17.482760,2.177780,4.172762,0.266121,0.556698,0.128074


In [18]:
df_predictions_MDCK = pd.DataFrame( models['Extra_Trees'].predict(X_test), 
                              index=test_MDCK['Molecule Name'], 
                              columns=['MDCK']).reset_index()
print(df_predictions_MDCK.head())

  Molecule Name       MDCK
0  ASAP-0014619  13.618269
1  ASAP-0014632  15.208748
2  ASAP-0014717  14.927788
3  ASAP-0014745  14.348529
4  ASAP-0014750  13.819529


/home/lic154/myapps/anaconda3/envs/atfp/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  warnings.warn(


# ML training for LogD

In [19]:
task = 'logD'

fp_logD_test = pd.read_csv('../data_my/fp_admet/dissect_logD_test.tsv', sep=r'\t|\s+', engine='python', header=None )
fp_logD_train = pd.read_csv('../data_my/fp_admet/dissect_logD_train.tsv', sep =r'\s+', engine='python', header=None)

fp_column_name =  ['fp_' + str(i+1) for i in range(len(fp_logD_test.columns)-1 )]

fp_logD_test.columns = ['Molecule Name'] + fp_column_name
fp_logD_train.columns =['Molecule Name'] + ['outputs'] + fp_column_name


tmp_logD_train = pd.merge( fp_logD_train, combined_df, on='Molecule Name', how='left')
train_logD = tmp_logD_train.drop(combined_df.columns[[0, 1, 2, 3, 4, 6, 7]], axis=1) 

tmp_logD_test = pd.merge(fp_logD_test, combined_df,  on='Molecule Name', how='left')
test_logD = tmp_logD_test.drop(combined_df.columns[[0, 1, 2, 3, 4, 6, 7]], axis=1) 

train_logD.dropna(inplace=True)
test_logD.dropna(inplace=True)

X_train = train_logD.iloc[:,2:]
y_train = train_logD['outputs']

X_test = test_logD.iloc[:,1:]


X_train.shape, X_test.shape

((352, 131), (126, 131))

In [20]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
# 归一化处理
scaler = MinMaxScaler()
#scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train)  # 用训练数据拟合并转换
# X_test = scaler.transform(X_test)  # 用相同的缩放器转换测试数据

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score

metric_dict = {}

for name, model in models.items():
    print(f"Processing model: {name}")
    
    # 基于交叉验证计算各项指标（注意：sklearn的负值评分，所以需要取负）
    cv_mse = cross_val_score(model, X_train, y_train, cv=cv_strategy, scoring='neg_mean_squared_error')
    cv_mae = cross_val_score(model, X_train, y_train, cv=cv_strategy, scoring='neg_mean_absolute_error')
    cv_r2  = cross_val_score(model, X_train, y_train, cv=cv_strategy, scoring='r2')
    
    # 计算每个CV折中的RMSE（注意：cv_mse为负值）
    cv_rmse = np.sqrt(-cv_mse)
    
    # 如果需要，也可以在这里对整个训练集进行模型拟合（但不参与指标计算）
    model.fit(X_train, y_train)
    
    # 保存交叉验证指标的均值和标准差（作为变异性的度量）
    metric_dict[name] = {
        "CV MAE Mean": -np.mean(cv_mae),
        "CV MAE Std": np.std(cv_mae),
        "CV MSE Mean": -np.mean(cv_mse),
        "CV MSE Std": np.std(cv_mse),
        "CV RMSE Mean": np.mean(cv_rmse),
        "CV RMSE Std": np.std(cv_rmse),
        "CV R2 Mean": np.mean(cv_r2),
        "CV R2 Std": np.std(cv_r2)
    }
    
    print(f"Completed processing model: {name}\n")

# 将结果字典转换为DataFrame以便更好地查看
results_df = pd.DataFrame(metric_dict).T  # 转置，使得每一行对应一个模型
print(results_df)

Processing model: Linear
Completed processing model: Linear

Processing model: Ridge
Completed processing model: Ridge

Processing model: Lasso
Completed processing model: Lasso

Processing model: RF
Completed processing model: RF

Processing model: SVR
Completed processing model: SVR

Processing model: DT
Completed processing model: DT

Processing model: GraBoost
Completed processing model: GraBoost

Processing model: AdaBoost
Completed processing model: AdaBoost

Processing model: XGBoost


Completed processing model: XGBoost

Processing model: Extra_Trees
Completed processing model: Extra_Trees

Processing model: KNN
Completed processing model: KNN

Processing model: ElasticNet
Completed processing model: ElasticNet

Processing model: Stacking
Completed processing model: Stacking

Processing model: Bagging
Completed processing model: Bagging

Processing model: Bayesian Ridge
Completed processing model: Bayesian Ridge

Processing model: SGD
Completed processing model: SGD

                 CV MAE Mean    CV MAE Std   CV MSE Mean    CV MSE Std  \
Linear          4.129753e+10  6.688380e+10  2.884530e+23  5.441431e+23   
Ridge           4.075707e-01  1.543094e-02  2.987445e-01  3.015911e-02   
Lasso           1.011328e+00  7.173833e-02  1.515925e+00  1.572426e-01   
RF              3.850313e-01  4.165520e-02  2.769507e-01  2.959296e-02   
SVR             4.129644e-01  2.315235e-02  3.137541e-01  3.751621e-02   
DT              5.173565e-01  6.834911e-02  5.897549e-01  9.7059

In [22]:
results_df.sort_values('CV RMSE Mean', ascending=True).head()

,CV MAE Mean,CV MAE Std,CV MSE Mean,CV MSE Std,CV RMSE Mean,CV RMSE Std,CV R2 Mean,CV R2 Std
Extra_Trees,0.347963,0.036053,0.238129,0.032625,0.486766,0.034456,0.838056,0.023138
GraBoost,0.365218,0.024584,0.250983,0.023332,0.500429,0.023532,0.827497,0.031053
Stacking,0.392070,0.016419,0.275338,0.018160,0.524432,0.017572,0.811796,0.024307
RF,0.385031,0.041655,0.276951,0.029593,0.525517,0.027979,0.810458,0.028902
XGBoost,0.378895,0.037949,0.286656,0.035260,0.534307,0.034242,0.804633,0.028845


In [23]:
results_df.sort_values('CV R2 Mean', ascending=False).head()

,CV MAE Mean,CV MAE Std,CV MSE Mean,CV MSE Std,CV RMSE Mean,CV RMSE Std,CV R2 Mean,CV R2 Std
Extra_Trees,0.347963,0.036053,0.238129,0.032625,0.486766,0.034456,0.838056,0.023138
GraBoost,0.365218,0.024584,0.250983,0.023332,0.500429,0.023532,0.827497,0.031053
Stacking,0.392070,0.016419,0.275338,0.018160,0.524432,0.017572,0.811796,0.024307
RF,0.385031,0.041655,0.276951,0.029593,0.525517,0.027979,0.810458,0.028902
XGBoost,0.378895,0.037949,0.286656,0.035260,0.534307,0.034242,0.804633,0.028845


In [24]:
df_predictions_logD = pd.DataFrame( models['Extra_Trees'].predict(X_test), 
                              index=test_logD['Molecule Name'], 
                              columns=['logD']).reset_index()
print(df_predictions_logD.head())

  Molecule Name    logD
0  ASAP-0014619  2.5153
1  ASAP-0014632  2.5110
2  ASAP-0014717  2.5968
3  ASAP-0014745  2.4783
4  ASAP-0014750  2.5025


/home/lic154/myapps/anaconda3/envs/atfp/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but ExtraTreesRegressor was fitted without feature names
  warnings.warn(


In [25]:
# df_KSOL_best =pd.read_csv("/home/lic154/data/AttentiveFP-master/code_admet_challenge/outputs/LC_test_pred_adme+30embs.csv").iloc[:,1:3]

In [26]:
df_pred_0225 = pd.merge(df_predictions_KSOL, df_predictions_logD, on="Molecule Name").merge(df_predictions_MDCK,on="Molecule Name")

In [27]:

df_pred_0225.to_csv("./outputs/LC_0225.csv")